In [7]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from lightgbm import LGBMClassifier
#from lightgbm import LGBMRegressor

import numpy as np
import pandas as pd
import os
import pickle
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import StratifiedKFold

import statsmodels.api as sm
import json
from sklearn.feature_selection import RFE

from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from random import choice
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# Lectura de vista minable

In [8]:
dataset = pd.read_csv("data/dataset_final.csv")
dataset.head(5)

,user_id,qty_adopciones_1M,qty_adopciones_3M,qty_adopciones_6M,qty_adopciones_9M,mes,anio,qty_dias_ultima_adopcion,qty_dias_ultimo_page_ingreso,snapshot_mes_x,...,ratio_adoption_preferencia_CONNECTION_SPEED_2.0,ratio_adoption_preferencia_CONNECTION_SPEED_3.0,ratio_adoption_interes_CONNECTION_SPEED,snapshot_mes,ratio_adoption_preferencia_ON_SITE_SEARCH_TERM_1,ratio_adoption_preferencia_ON_SITE_SEARCH_TERM_1.0,ratio_adoption_preferencia_ON_SITE_SEARCH_TERM_63.0,ratio_adoption_preferencia_ON_SITE_SEARCH_TERM_89.0,ratio_adoption_interes_ON_SITE_SEARCH_TERM,target
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,9,...,-999.0,0.0,0.0,9,0.0,-999.0,-999.0,-999.0,0.0,0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,9,...,0.0,-999.0,0.0,9,0.0,-999.0,-999.0,-999.0,0.0,0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,9,...,0.0,0.0,0.0,9,0.0,-999.0,-999.0,-999.0,0.0,0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,9,...,-999.0,-999.0,0.0,9,0.0,-999.0,-999.0,-999.0,0.0,0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,9,...,0.0,0.0,0.0,9,0.0,-999.0,-999.0,-999.0,0.0,0


In [21]:
dataset = dataset.drop(['FEC_EVENT_MAX','FEC_EVENT_MENOS_7D'], axis = 1)

KeyError: "['FEC_EVENT_MAX' 'FEC_EVENT_MENOS_7D'] not found in axis"

In [17]:
dataset = dataset.replace(np.inf, 999)
dataset = dataset.fillna(999)

In [22]:
train_features = dataset.drop(['target'], axis = 1)
train_labels = dataset.target

# Training

In [10]:
def crossValidationSelector(X, y, folds):  
    skf = StratifiedKFold(n_splits=folds)
    return skf.split(X, y)

In [11]:
def trainWithCrossValidation(kfold, base, learner):    
    # kfolds are the folds that return crossValidationSelector
    # base is the dataset
    # learner is the model with the best parameters
    proba = pd.DataFrame()
    preds = pd.DataFrame()
    X, y = base.drop(base.columns[-1], axis=1), base.iloc[:,-1] 
    
    for train_index, test_index in kfold:
#    for train_index, test_index in kfold.split(X, y):
        X_train, X_test = X.loc.__getitem__(train_index), X.loc.__getitem__(test_index)
        y_train, y_test = y.loc.__getitem__(train_index), y.loc.__getitem__(test_index)    
        auxTest = X_test.copy()
        X_train = X_train.values.astype(np.float32, order="C")
        X_test = X_test.values.astype(np.float32, order="C")
        scaler = StandardScaler()
        scaler.fit(np.r_[X_train, X_test])
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

    
        probabilidades, predicciones = train(learner, X_train, y_train, X_test, y_test)     
        auxTest['prob'] = probabilidades
        proba = pd.concat([proba, auxTest[['prob']]])
        
        auxTest['pred'] = predicciones
        preds = pd.concat([preds, auxTest[['pred']]])
        
    #fold_importance_df = pd.DataFrame()
    #fold_importance_df["feature"] = X.columns
    #fold_importance_df["importance"] = learner.feature_importance()
    #feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    return proba, preds, learner

#####################################################################################################################


def train(learner, X_train, y_train, X_test, y_test, esr=20):
    mask = y_test.isin(y_train.unique()).values
    learner.fit(X_train, y_train,
                eval_set=[(X_test, y_test),
                          (X_test[mask], y_test[mask])
                          ],
                early_stopping_rounds=esr,
                verbose=True)
    #n_estimators = learner.best_iteration
    n_estimators = 10000
    probs = learner.predict_proba(X_test, ntree_limit=n_estimators)[:, -1]
    pred = learner.predict(X_test, ntree_limit=n_estimators)
    
    
    
    return probs, pred

# Metrics

In [12]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    accuracy = metrics.accuracy_score(test_labels, predictions)
    return accuracy

# Tunning

<h6>Random search de los mejores parametros</h6>

In [13]:
# Se le pasa el modelo, un grid de parametros, los features y labels para entrenar
def trainRandomGrid(candidate, random_grid, train_features, train_labels):
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    model = candidate
    # Random search of parameters, using 3 fold cross validations
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 5, cv = 5, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    
    rf_random.fit(X = train_features, y= train_labels)
    
    # We can view the best parameters from fitting the random search
    # rf_random.best_params_

    return rf_random

<h6>Grid search a partir de Random search en busqueda de los mejores parametros</h6>

In [14]:
def trainGrid(candidate, param_grid, train_features, train_labels):
    # Create a based model
    model = candidate
    # Instantiate the grid search model
    grid_search = GridSearchCV(estimator = model, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

    ######## TRAIN GRID SELECTOR #######
    # Fit the grid search to the data
    grid_search.fit(train_features, train_labels)
    #grid_search.best_params_
    return grid_search

# Ejecucion

<h6>Buscamos los mejores parametros de manera random para achicar el espacio de busqueda</h6>

In [ ]:

model = LGBMClassifier()

params = {
    'learning_rate': [0.05],
    'num_leaves': [10],
    'boosting_type' : ['gbdt','dart'],
    'objective' : ['binary'],
    'max_depth' : [5],
    'random_state' : [501], 
    'colsample_bytree' : [0.5],
    'subsample' : [0.5],
    'min_split_gain' : [0.01],
    'min_data_in_leaf':[10],
    'metric':['auc']
    }


randomGrid = trainRandomGrid(model, params, train_features, train_labels)
best_random = randomGrid.best_estimator_

#Print best paramas and accuracy
best_random = randomGrid.best_estimator_
random_accuracy = evaluate(best_random, train_features, train_labels)

print(randomGrid.best_estimator_)
print(random_accuracy)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


/Users/lbatlle/.virtualenvs/pythonenv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


<h6>Ahora con un espacio de busqueda mas acotado realizamos una busqueda de mejores paramestros de manera exhaustiva</h6>

In [ ]:
new_params = { "params": {
                        'learning_rate': [0.05],
                        'num_leaves': [90,200,500],
                        'boosting_type' : ['gbdt','dart'],
                        'objective' : ['binary'],
                        'max_depth' : [9,10,11,15,20,25],
                        'random_state' : [501,600,750], 
                        'colsample_bytree' : [0.5,0.7,0.10,0.20],
                        'subsample' : [0.5,0.7,0.3,0.1,0.9],
                        'min_split_gain' : [0.001,0.01,0.05,0.10],
                        'min_child_samples' : [0.001,0.005,0.010,0.0001],
                        'min_data_in_leaf':[10],
                        'metric':['auc']
                        }
             }


In [ ]:
params = new_params["params"]
bestGrid = trainGrid(model, params, train_features, train_labels)

#Evaluate the model
best_model = bestGrid.best_estimator_
accuracy = evaluate(best_model, train_features, train_labels)

#Print best paramas and accuracy
print(bestGrid.best_estimator_)
print(accuracy)

<h6>Teniendo los mejores hiperparametros procedemos a entrenar el modelo </h6>

In [ ]:
model = LGBMClassifier()

kfolds = crossValidationSelector(train_features, train_labels, 5)
proba, preds, model = trainWithCrossValidation(kfolds, dataset, model)


<h6>Vemos la importancia de las variables</h6>

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

<h6>Por ultimo vemos las metricas resultantes</h6>

In [ ]:
### AUROC ###

fpr, tpr, thresholds = metrics.roc_curve(np.asarray(dataset.target), np.asarray(preds))
m = {"auc": metrics.roc_auc_score(np.asarray(dataset.target), np.asarray(preds)),
                    "tpr": tpr,
                    "fpr": fpr                        
                    }    

    
label = 'ROC curve for {0}:'.format(m)
plt.plot(m['fpr'], m['tpr'], label=label+'AUC={0:0.2f}'.format(m['auc']))

plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
#
#
#
#
plt.grid(True)
plt.title('ROC')
plt.legend(loc="lower left")
plt.show()

In [ ]:
### MATRIZ DE CONFUSION ###

confusion_mc = confusion_matrix(train_labels,preds)
df_cm = pd.DataFrame(confusion_mc)

plt.figure(figsize=(5.5,4))
sns.heatmap(df_cm, annot=True)
plt.title('XGBoost RBF Kernel \nAccuracy: {0:.3f}'
        .format(accuracy_score(train_labels,preds)))
plt.ylabel('True label')
plt.ylabel('Predicted label')